## Notas


### Asignamiento de recursos de CPU y memoria a cada pod

Pendiente
https://www.youtube.com/watch?v=xTTJg1aJ4kg&list=PLqRCtm0kbeHA5M_E_Anwu-vh4NWlgrOY_&index=4


### HPA
Buscar sobre HPA (Horizontal pod autoscaling), se encarga de crear reglas para determinar la cantidad de pods, por ejemplo según uso de CPU o cantidad de memoria utilizada.
https://www.youtube.com/watch?v=T6wRsmrm_gk&list=PLqRCtm0kbeHA5M_E_Anwu-vh4NWlgrOY_&index=5


### LIVENESS Y READINESS PROBES

https://www.youtube.com/watch?v=5gSc1ouW8rM&list=PLqRCtm0kbeHA5M_E_Anwu-vh4NWlgrOY_&index=6

# Kubernetes

Dada la problemática que presenta la escabilidad en entornos de múltiples servidores, se creó Kubernetes para permitir una administración de estos, permitiendo automatizar el despliegue, ajustar la escala y ayudar con el manejo de contenedores

### Un ejemplo

Si durante la noche un servidor sufre un desperfecto, kubernetes se encargará de montar los contenedores perdidos en otros servidores para mantener el sistema funcionando

Kubernetes tiene tres partes muy importantes:
- Controladores: Se preocupan de decidir donde estarán los contenedores, los requerimientos que necesita cada uno.
- Workers: Máquinas que ejecutan pods, los nodos trabajan en red, es decir, si un nodo cae no importa, el controlador puede levantar los pods perdidos en otra máquina y la red continúa trabajando.
- etcd: Aquí se almacena la información del estado de elemento miembro del clúster.

<img src="images/arquitectura" width="800" height="800" align="center"/>


#### Kubelet
Agente que se ejecuta en cada nodo de un clúster. Se asegura de que los contenedores estén corriendo en un pod.  
El agente kubelet toma un conjunto de especificaciones de Pod, llamados PodSpecs, que han sido creados por Kubernetes y garantiza que los contenedores descritos en ellos estén funcionando y en buen estado.

#### Scheduler
Componente del plano de control que está pendiente de los Pods que no tienen ningún nodo asignado y seleciona uno donde ejecutarlo.  
Para decidir en qué nodo se ejecutará el pod, se tienen en cuenta diversos factores: requisitos de recursos, restricciones de hardware/software/políticas, afinidad y anti-afinidad, localización de datos dependientes, entre otros.

#### Controller-manager

Componente del plano de control que ejecuta los controladores de Kubernetes.

Lógicamente cada controlador es un proceso independiente, pero para reducir la complejidad, todos se compilan en un único binario y se ejecuta en un mismo proceso.

<img src="images/arquitectura" width="800" height="800" align="center"/>

<font size="3">Estos controladores incluyen:</font>
  
   **<font size="3">Controlador de nodos</font>**<font size="3">: es el responsable de detectar y responder cuándo un nodo deja de funcionar.</font>  
   **<font size="3">Controlador de replicación</font>**<font size="3">: es el responsable de mantener el número correcto de pods para cada controlador de replicación del sistema.</font>  
   **<font size="3">Controlador de endpoints</font>**<font size="3">: construye el objeto Endpoints, es decir, hace una unión entre los Services y los Pods.</font>  
   **<font size="3">Controladores de tokens y cuentas de servicio</font>**<font size="3">: crean cuentas y tokens de acceso a la API por defecto para los nuevos Namespaces.</font>

# Montando un clúster básico con Minikube



Chequear que la virtualización está habilitada en la bios del pc:  
```egrep --color 'vmx|svm' /proc/cpuinfo```

Instalar un operador de maquinas virtuales (VirtualBox, KVM, etc)  

Instalar kubectl:
```sudo pacman -S kubectl```

Instalar minikube:
```sudo pacman -S minikube```


In [1]:
minikube start

😄  minikube v1.13.1 en Arch 20.1.1
✨  Using the kvm2 driver based on existing profile
👍  Starting control plane node minikube in cluster minikube
🔄  Restarting existing kvm2 VM for "minikube" ...
🐳  Preparando Kubernetes v1.19.2 en Docker 19.03.12...
🔎  Verifying Kubernetes components...
🔎  Verifying ingress addon...
🌟  Enabled addons: default-storageclass, ingress, storage-provisioner
🏄  Done! kubectl is now configured to use "minikube" by default


In [2]:
kubectl get nodes

NAME       STATUS   ROLES    AGE   VERSION
minikube   Ready    master   15h   v1.19.2


In [32]:
minikube status

minikube
type: Control Plane
host: Running
kubelet: Running
apiserver: Running
kubeconfig: Configured



## Conceptos



### Namespace  
Básicamente crean clústers virtuales dentro del mismo clúster físico, ocupan la misma infraestructura pero cada uno permanece aislado del otro, por ejemplo esto es ideal para tener clústers de desarrollo y producción.  

In [33]:
kubectl get namespaces

NAME              STATUS   AGE
default           Active   176m
kube-node-lease   Active   177m
kube-public       Active   177m
kube-system       Active   177m
testing           Active   174m


### Pods  

Es la unidad mas pequeña que maneja kubernetes, consiste en un conjunto de contenedores donde funciona alguna aplicación.  

#### Consideraciones
No use Pods desnudos (es decir, Pods que no estén vinculados a ReplicaSet o Deployment) si puede evitarlo. Los pods desnudos no se reprogramarán en caso de falla de un nodo.  

Una deployment crea un ReplicaSet para garantizar que la cantidad deseada de pods siempre esté disponible, y especifica una estrategia para reemplazar los pods (como RollingUpdate), es casi siempre preferible a crear pods directamente. Un job también puede ser apropiado.

In [34]:
kubectl get pods

NAME                     READY   STATUS    RESTARTS   AGE
hello-868bcb8b84-jndvc   1/1     Running   0          172m
hello-868bcb8b84-rdd5h   1/1     Running   0          172m
hello-868bcb8b84-xtft5   1/1     Running   0          172m


### Servicios  

Es una abstracción que define un conjunto de pods que implementan un micro-servicio.  

Existen 3 tipos de servicios:



ClusterIP:  
Solo permite el acceso interno entre distintos servicios. Es el tipo por defecto. Podemos acceder desde el exterior con la instrucción kubectl proxy, puede de ser gran ayuda para los desarrolladores.  

![title](images/ClusterIP)

NodePort:  
Abre un puerto, para que el servicio sea accesible desde el exterior. Por defecto el puerto generado está en el rango de 30000:40000. Para acceder usamos la ip del servidor master del cluster y el puerto asignado.

![title](images/NodePort)

LoadBalancer:  
Este tipo sólo esta soportado en servicios de cloud público (GKE, AKS o AWS). El proveedor asignara un recurso de balanceo ed carga para el acceso a los servicios. si usamos un cloud privado, como OpenSatck necesitaremos un plugin para configurar el funcionamiento.

![title](images/LoadBalancer)

#### Consideraciones
Define y usa etiquetas que identifiquen atributos semanticos de tu aplicación o deployment, por ejemplo:
```{ app: myapp, tier: frontend, phase: test, deployment: v3 }```. Puedes usar estas etiquetas para seleccionar apropiadamente los pods para otros recursos; por ejemplo, si quiero que un servicio seleccione todos los pods ```tier: frontend``` o todos los componentes ```phase: test``` de la ```app: myapp```.

In [35]:
kubectl get services

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   176m


### Manifiestos  

Los recursos pueden crearse directamente desde la línea de comandos, pero esto no es recomendable en producción debido a los potenciales errores humanos al realizar estas tareas, por esta razón es mejor utilizar los manifiestos.

Los manifiestos son documentos utilizados para crear, modificar o eliminar recursos de Kubernetes, como pods, despliegues, servicios e ingresos.

In [36]:
cat yaml/00-namespace.yaml

kind: Namespace
apiVersion: v1
metadata:
  name: testing 


In [37]:
kubectl apply -f yaml/00-namespace.yaml

namespace/testing unchanged


In [38]:
kubectl get namespaces

NAME              STATUS   AGE
default           Active   176m
kube-node-lease   Active   177m
kube-public       Active   177m
kube-system       Active   177m
testing           Active   175m


In [39]:
cat yaml/01-wordpress-service.yaml

#Por norma general, todos los manifiestos deben llevar:
#   -Encabezado
#   -Metadatos
#   -Especificaciones

kind: Service           #pod, deployment, service, namespace, replicaset, StatefulSet, DaemonSet, 
                        #Job o lo que sea que vayas a describir
apiVersion: v1
metadata:
  name: wordpress       #nombre del servicio en este caso
spec:
  type: NodePort        #Aquí se indica si se trabajará con las ip internas o se abrirá un puerto 
                        #para desplegarlo externamente, según vimos, NodePort ordena abrir un puerto
                        #en el worker para poder acceder externamente a este servicio
  ports:
  - port: 80             #Se indica el puerto que se abre en el worker
    targetPort: 80      #Es el puerto al que apunta el servicio en cualquier pod que tenga el rol
                        #wordpress
    nodePort: 30000     #Es el puerto del nodo que apunta al 80 del pod
  selector:
    role: wordpress

# Si se desea, se puede especifica

In [40]:
kubectl -n testing apply -f yaml/01-wordpress-service.yaml
kubectl -n testing get services

service/wordpress unchanged
NAME        TYPE       CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
wordpress   NodePort   10.105.93.115   <none>        80:30000/TCP   175m


In [41]:
cat yaml/02-wordpress-rc.yaml

apiVersion: v1
kind: ReplicationController
metadata:
  name: wordpress
spec:
  replicas: 2
  template:
    metadata:
      labels:
        role: wordpress
    spec:
      containers:
      - name: wordpress
        image: wordpress:php7.2-apache
        imagePullPolicy: IfNotPresent
        ports:
        - containerPort: 80


La imagen se descarga por defecto desde DockerHub; si quieres usar un registry alternativo, debes indicar la URL completa al recurso.

In [42]:
kubectl -n testing apply -f yaml/02-wordpress-rc.yaml
kubectl -n testing get pods

replicationcontroller/wordpress unchanged
NAME              READY   STATUS    RESTARTS   AGE
wordpress-hsp8r   1/1     Running   0          173m
wordpress-t9ldz   1/1     Running   0          174m


In [44]:
kubectl -n testing delete pod wordpress-hsp8r
kubectl -n testing get pods

pod "wordpress-hsp8r" deleted
NAME              READY   STATUS              RESTARTS   AGE
wordpress-smhzw   0/1     ContainerCreating   0          20s
wordpress-t9ldz   1/1     Running             0          175m


Ahora podemos ver que nuestro servicio funciona correctamente

In [45]:
kubectl get nodes -o wide

NAME       STATUS   ROLES    AGE    VERSION   INTERNAL-IP      EXTERNAL-IP   OS-IMAGE               KERNEL-VERSION   CONTAINER-RUNTIME
minikube   Ready    master   178m   v1.19.2   192.168.39.143   <none>        Buildroot 2019.02.11   4.19.114         docker://19.3.12


In [46]:
kubectl -n testing get services

NAME        TYPE       CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
wordpress   NodePort   10.105.93.115   <none>        80:30000/TCP   175m


In [47]:
minikube service -n testing wordpress --url

http://192.168.39.143:30000


## Deployments y ReplicaSets

#### Deployment:
Un controlador de Deployment proporciona actualizaciones declarativas para los Pods y los ReplicaSets.

Cuando describes el estado deseado en un objeto Deployment, el controlador del Deployment se encarga de cambiar el estado actual al estado deseado de forma controlada. Puedes definir Deployments para crear nuevos ReplicaSets, o eliminar Deployments existentes y adoptar todos sus recursos con nuevos Deployments.

#### ReplicaSets:
El objeto de un ReplicaSet es el de mantener un conjunto estable de réplicas de Pods ejecutándose en todo momento. Así, se usa en numerosas ocasiones para garantizar la disponibilidad de un número específico de Pods idénticos.

In [48]:
cat yaml/01-hello-app-deployment.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  name: hello
spec:
  replicas: 3
  selector:
    matchLabels:
      role: hello
  template:
    metadata:
      labels:
        role: hello
    spec:
      containers:
      - name: hello
        image: gcr.io/google-samples/hello-app:1.0
        imagePullPolicy: IfNotPresent
        ports:
        - containerPort: 8080

        
        


In [49]:
kubectl apply -f yaml/01-hello-app-deployment.yaml

deployment.apps/hello unchanged


Estado actual tanto del Namespace testing como del default

In [50]:
kubectl -n testing get all

NAME                  READY   STATUS    RESTARTS   AGE
pod/wordpress-smhzw   1/1     Running   0          2m29s
pod/wordpress-t9ldz   1/1     Running   0          177m

NAME                              DESIRED   CURRENT   READY   AGE
replicationcontroller/wordpress   2         2         2       177m

NAME                TYPE       CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
service/wordpress   NodePort   10.105.93.115   <none>        80:30000/TCP   177m


In [51]:
kubectl -n default get all

NAME                         READY   STATUS    RESTARTS   AGE
pod/hello-868bcb8b84-jndvc   1/1     Running   0          175m
pod/hello-868bcb8b84-rdd5h   1/1     Running   0          175m
pod/hello-868bcb8b84-xtft5   1/1     Running   0          175m

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   179m

NAME                    READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/hello   3/3     3            3           176m

NAME                               DESIRED   CURRENT   READY   AGE
replicaset.apps/hello-868bcb8b84   3         3         3       175m


Si lo deseamos podemos aumentar la cantidad de pods para un deployment, para así asignar mas recursos a este.

In [53]:
kubectl scale --replicas=5 deployment/hello

deployment.apps/hello scaled


In [54]:
kubectl -n default get all

NAME                         READY   STATUS              RESTARTS   AGE
pod/hello-868bcb8b84-gtb5x   0/1     ContainerCreating   0          11s
pod/hello-868bcb8b84-jndvc   1/1     Running             0          177m
pod/hello-868bcb8b84-rdd5h   1/1     Running             0          177m
pod/hello-868bcb8b84-w8rmx   0/1     ContainerCreating   0          11s
pod/hello-868bcb8b84-xtft5   1/1     Running             0          177m

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   3h

NAME                    READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/hello   3/5     5            3           177m

NAME                               DESIRED   CURRENT   READY   AGE
replicaset.apps/hello-868bcb8b84   5         5         3       177m


Se crea el servicio hello para permitir el acceso externo a la aplicación

In [61]:
cat yaml/02-hello-app-service-node-port.yaml

apiVersion: v1
kind: Service
metadata:
  name: hello
spec:
  type: NodePort
  ports:
  - port: 8080
    targetPort: 8080
    nodePort: 30001
  selector:
    role: hello


In [71]:
kubectl apply -f yaml/02-hello-app-service-node-port.yaml

service/hello created


In [72]:
kubectl describe service hello

Name:                     hello
Namespace:                default
Labels:                   <none>
Annotations:              <none>
Selector:                 role=hello
Type:                     NodePort
IP:                       10.100.218.209
Port:                     <unset>  8080/TCP
TargetPort:               8080/TCP
NodePort:                 <unset>  30001/TCP
Endpoints:                172.17.0.4:8080,172.17.0.5:8080,172.17.0.6:8080 + 2 more...
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>


In [73]:
minikube service hello --url

http://192.168.39.143:30001


Ahora en nuestro deployment con el rol "hello" existirán siempre 5 pods corriendo gracias al replicaSet que se generó para supervisar esto, los que consiguieron exponerse al exterior gracias al servicio que acabamos de crear en el puerto 30001.

Entonces si tenemos 5 replicas de nuestro contenedor corriendo bajo el mismo puerto externo, podemos ver como trabaja el balanceador enviando consultas a este puerto:

In [74]:
curl http://192.168.39.143:30001
curl http://192.168.39.143:30001
curl http://192.168.39.143:30001
curl http://192.168.39.143:30001
curl http://192.168.39.143:30001

Hello, world!
Version: 1.0.0
Hostname: hello-868bcb8b84-xtft5
Hello, world!
Version: 1.0.0
Hostname: hello-868bcb8b84-gtb5x
Hello, world!
Version: 1.0.0
Hostname: hello-868bcb8b84-rdd5h
Hello, world!
Version: 1.0.0
Hostname: hello-868bcb8b84-gtb5x
Hello, world!
Version: 1.0.0
Hostname: hello-868bcb8b84-gtb5x


In [78]:
kubectl -n default get replicaset
kubectl -n testing get rs

NAME               DESIRED   CURRENT   READY   AGE
hello-868bcb8b84   5         5         5       4h11m
No resources found in testing namespace.


Aquí podemos ver que para el namespace default se generó un ReplicaSet llamado "hello-868bcb8b84", pero para el namespace testing no existe ninguno ¿A qué se debe esto?

Esto se debe a que si recordamos, solo generamos un deployment para "hello".
Al generar un deployment, este manda a crear un ReplicaSet para que supervise que siempre se mantengan los pods según las instrucciones que le dimos en el manifiesto, es decir, se preocupa de velar por que se respeten las ordenes generadas en el deployment.

Si eliminamos un pod, el ReplicaSet se encarga de tomar el modelo de pod especificado y crear replicas hasta que se cumpla el total de replicas que le encomendamos, ojo que si modificamos la etiqueta de un pod, el replicaset también creará otra copia para mantener la integridad.

Nota: En caso de detectar un pod con mal funcionamiento es recomendable no destruirlo sin averiguar la razón, en este caso es necesario apartarlo del tráfico regular, esto se hace facilmente solo cambiándole la etiqueta para que el ReplicaSet se encargue de generar una nueva copia pero sin eliminar el pod en mal estado.

## Actualizar versiones

Para actualizar de una versión de imagen a otra, es cosa de aplicar otro manifiesto identico al de la versión prueba pero con la imagen de versión actualizada, esto le dará la orden al replicaSet de realizar los cambios respectivos para matar los pods existentes y generar los nuevos

In [80]:
cat yaml/01-hello-app-deployment.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  name: hello
spec:
  replicas: 3
  selector:
    matchLabels:
      role: hello
  template:
    metadata:
      labels:
        role: hello
    spec:
      containers:
      - name: hello
        image: gcr.io/google-samples/hello-app:1.0
        imagePullPolicy: IfNotPresent
        ports:
        - containerPort: 8080

        
        


In [79]:
cat yaml/03-hello-app-deployment.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  name: hello
spec:
  replicas: 3
  selector:
    matchLabels:
      role: hello
  template:
    metadata:
      labels:
        role: hello
    spec:
      containers:
      - name: hello
        image: gcr.io/google-samples/hello-app:2.0
        imagePullPolicy: IfNotPresent
        ports:
        - containerPort: 8080


In [81]:
kubectl apply -f yaml/03-hello-app-deployment.yaml

deployment.apps/hello configured


In [82]:
kubectl get pods

NAME                     READY   STATUS        RESTARTS   AGE
hello-6c7b794697-82ccg   1/1     Running       0          32s
hello-6c7b794697-ckbt5   0/1     Pending       0          6s
hello-6c7b794697-lz6hg   1/1     Running       0          80s
hello-868bcb8b84-rdd5h   1/1     Terminating   0          4h28m
hello-868bcb8b84-xtft5   1/1     Running       0          4h28m


In [83]:
kubectl get pods

NAME                     READY   STATUS    RESTARTS   AGE
hello-6c7b794697-82ccg   1/1     Running   0          92s
hello-6c7b794697-ckbt5   1/1     Running   0          66s
hello-6c7b794697-lz6hg   1/1     Running   0          2m20s


## Dashboard


In [3]:
minikube dashboard

🔌  Enabling dashboard ...
🤔  Verifying dashboard health ...
🚀  Launching proxy ...
🤔  Verifying proxy health ...
🎉  Opening http://127.0.0.1:42935/api/v1/namespaces/kubernetes-dashboard/services/http:kubernetes-dashboard:/proxy/ in your default browser...

